In [1]:
from random import randint
from hashlib import sha3_256



class DOT():
    def __init__(self, x, y, p=None):
        self.x = x
        self.y = y
        self.p = p

    @property
    def info(self):
        print(self.x, self.y)

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"

class EC():
    def __init__(self, a, b, p) -> None:
        #y^2 = x^3 + a*x +b mod p
        self.a = a
        self.b = b
        self.p = p

    def dop_info(self, n,h) -> None:
        self.n = n
        self.h = h

    def add_seed(self, dot):
        self.seed = dot

    def modInverse(self, x):
        return self.p - x

    def add(self, a, b):
        g = DOT(a.x, a.y)
        dx = (a.x - b.x) % self.p
        dy = (a.y - b.y) % self.p
        m = dy * (-dx % self.p) % self.p
        g.x = (m*m -a.x -b.x) % self.p
        g.y = (m * (a.x - g.x) - a.y) % self.p
        return g

    def __mul__(self,  N):
        #if a is None:
        a = DOT(self.seed.x, self.seed.y)
        b = DOT(a.x,a.y)
        for i in range(N):
            b = self.add(a,b)
        return b    

    def mul(self, a, N):
        #if a is None:
        #    a = DOT(self.seed.x, self.seed.y)
        b = DOT(a.x,a.y)
        for i in range(N):
            b = a + b
        return b 

    @property
    def info(self):
        print("y^2 = x^3 + x *(", a,") + (", b,") mod ", p)
        print("Порядок подгруппы: ", n)
        print("Кофактор подгруппы: ", h)
        print("seed: ", seed)

    def find_seed(self, seed = 0):
        if seed == 0:
            pass

class Crypt():
    def __init__(self, ec, start):
        self.ec = ec
        self.ec.seed=start
    
    def key_gen(self, priv):
        self.priv = priv
        hash_object = sha3_256(priv)
        hex_dig = hash_object.hexdigest()
        self.hash_priv = hex_dig, 16
        N = int(hex_dig, 16)%10**6
        self.pub = self.ec*N
        return self.pub
    
    @property
    def info(self):
        self.ec.info
        print("priv: ", self.priv)
        print("priv hash: ", self.hash_priv)
        print("pub: ", self.pub)

# В качестве параметров кривой взята первая рекомендация из NIST

In [2]:
a = -3
b = 2455155546008943817740293915197451784769108058161191238065
p = 6277101735386680763835789423207666416083908700390324961279
x = 602046282375688656758213480587526111916698976636884684818
y = 174050332293622031404857552280219410364023488927386650641 
n = 6277101735386680763835789423176059013767194773182842284081 
h = 1

curv = EC(a,b,p) 
curv.dop_info(n, h)
seed = DOT(x,y) 
curv.add_seed(seed)

k = randint(0,30)
start = curv * k
channel = Crypt(curv, start)
priv = b'Something'
pub = channel.key_gen(priv)
channel.info
#test = channel.key_gen(priv)
#print(pub.x == test.x and pub.y==test.y)
#print(pub)
#print(test)
#print(channel.key_gen(priv))

y^2 = x^3 + x *( -3 ) + ( 2455155546008943817740293915197451784769108058161191238065 ) mod  6277101735386680763835789423207666416083908700390324961279
Порядок подгруппы:  6277101735386680763835789423176059013767194773182842284081
Кофактор подгруппы:  1
seed:  (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
priv:  b'Something'
priv hash:  ('64e335d39d86e261cd0d4af21b0dc065e6f95003c110046a2f7d06a0c86c5008', 16)
pub:  (3764075828544895879853332447589608838424021119400635795754, 1795953031616290284564937229785190029368853224261098073492)


##Параметры ключа
#a,b,p,x,y
priv / pub
Все параметры занимают по 64 16-ых разряда

In [3]:
def fill(a,n, neg) -> str:
    while len(a)<n-1:
        a='0'+a
    if neg:
      return '1'+a
    else:
      return '0'+a

def to_16(n, N):
    neg = (type(n) is int) and (n < 0)
    n = abs(n)
    s = ''
    h = '0123456789ABCDEF'
    while n > 0:
        s = h[n % 16] + s
        n = n // 16
    return fill(s, N, neg)

def make_str(a):
    s=''
    for i in a:
        s = s+i
    return s

def make_arr(str):
    a=[]
    for i in range(len(str)//64):
        print(str[i*64:(i+1)*64])
        a.append(str[i*64:(i+1)*64])
    return a

#def re_fill(str):
#    neg = str[0] == 1:


In [4]:
a = channel.ec.a
b = channel.ec.b
p = channel.ec.p
x = channel.ec.seed.x
y = channel.ec.seed.y
#priv = channel.hash_priv[0]
params = [a,b,p,x,y]
priv = [to_16(i, 64) for i in params]
#print(len(channel.hash_priv[0]))
pub = list(priv)

priv.append(str(channel.hash_priv[0]))
pub.append(to_16(channel.pub.x, 64))
pub.append(to_16(channel.pub.y, 64))

#print(make_str(priv))
#print(make_str(pub))
#print(priv == make_arr(make_str(priv)))


1000000000000000000000000000000000000000000000000000000000000003000000000000000064210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B10000000000000000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFF0000000000000000163147DCF16F6EBDB65B4A69E689E7DAA1897E09809872DF00000000000000004ACD0A28E1FA81CA5056FBB452F0F9FB0548A23D2652A09B64e335d39d86e261cd0d4af21b0dc065e6f95003c110046a2f7d06a0c86c5008
1000000000000000000000000000000000000000000000000000000000000003000000000000000064210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B10000000000000000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFF0000000000000000163147DCF16F6EBDB65B4A69E689E7DAA1897E09809872DF00000000000000004ACD0A28E1FA81CA5056FBB452F0F9FB0548A23D2652A09B00000000000000009982C90A67C4087E69DCE6C5137BEFB73BECD28D8A24092A0000000000000000493E9FE2466D07EAA815C069328F9DED364D9F22E25EED94


In [7]:
file = open('key', 'w')
file.write(make_str(priv))
file.close()

In [8]:

file = open('key.pub', 'w')
file.write(make_str(pub))
file.close()